# How to use Deep Learning to predict Tomorrow's Temperature with Feedforward Network
## Univariate Multiple Linear Perceptron
### Author : Mohamed JELIDI, Email : jelidi.mohamad@gmail.com

A **feedforward neural network**, or FNN for short, is an artificial neural network wherein connections between nodes do not form a cycle. It is the first and simplest type of artificial neural network. 

Multilayer Perceptrons, or MLPs for short, are a class of FFN, a MLP consists of, at least, three layers of nodes: an input layer, a hidden layer and an output layer. Except for the input nodes, each node is a neuron that uses a nonlinear activation function.

MLP can be applied to time series forecasting, the challenge with utilizing MLPs for time series forecasting is in the data preparation or the preprocessing stage.
In this paper, you will discover how to develop a MLP model for a standard time series forecasting problem. 
When using one variable to train the model on your data, this type of models is called **Univariate time series forcasting**, therefore, the goal now is to show you :
- How to develop MLP models for univariate time series forecasting.

Multilayer Perceptrons, or MLPs for short, can be used to model **univariate time series forecasting problems**. Univariate time series are a dataset containing a single series of observations ordered chronologically and a model is required to learn from the series of past observations to predict the next value in the  sequence. This section is splitted into two subsections; they are:


1.   Preprocessing data
2.   Building the MLP Model



## Preprocessing data

Before building and training any model data should prepared so the univariate series can be modeled afterwards. The MLP model will learn a function that maps a sequence of past observations feeded as input to another output observation. As such, the sequence of observations must be transformed into several examples from which the model can learn. Consider a given univariate sequence:
[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
This is an example of a univariate timeseries, we have only one sequence of data out of which we would like to use it to create a model to predict something of its nature.

We can divide the sequence into various **input/output** patterns which we call *samples*, where three time steps are used as input and one time step is used as output for the one-step prediction that is being learned.

X,                      **y**

1, 2, 3,              **4**

5, 6, 7,              **8**

9, 10, 11,         **12**

...

It is always recommended to use a small public dataset that fits the memory of everyone, to demonstrate key concepts clearly with less hustle to get the data somewhere on the web. For this reason, we will be using **daily-minimum-temperatures.csv** this dataset contains Daily minimum temperatures in Melbourne, Australia, 1981-1990 and could be found easily on the web or you can download it from my github repository as well.


In [0]:
from pandas import Series
from pandas import DataFrame
from pandas import concat

We import **Series**, **DataFrame** and **concat** from **pandas**  nothing so special about that as we will need pandas to import the csv file, and concat which allows us to Concatenate pandas objects.

In [0]:
from google.colab import files
uploaded = files.upload()

Saving daily-minimum-temperatures.csv to daily-minimum-temperatures.csv


In [0]:
import io
series = Series.from_csv(io.BytesIO(uploaded['daily-minimum-temperatures.csv']), header=0)

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:2890: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


we used the Series method **from_csv(..)** to import the csv file, the phisical path as the first argument, and taking the first row as a header.

# Data Preparation

In [0]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

The **split sequence()** function above implements this behavior and will split a given univariate sequence into many samples where each sample has a specified number of *time steps* and the output is a single time step.

In [0]:
# split into samples
X, y = split_sequence(series, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

Now that we prepared our univariate series for modeling, let’s develop an MLP model that can learn the mapping between these inputs and its outputs.

Like I said in the introduction, A simple MLP model has a single hidden layer of nodes, and an output layer used to make a prediction. We can define an MLP for univariate time series forecasting as follows.

# MLP Model

In [0]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


The shape of the input is very important; because that is what the model expects as input for each sample in terms of the number of time steps.

The number of time steps as input is the number we chosen when we prepared our dataset as an argument to the **split sequence()** function, if we want to split the sample into three days to predict the forth day number of steps is then 3, if we want to use the data of a week to predict the next day number of steps is 7, and so on.

The input dimension for each sample is specified in the input dim argument on the definition of first hidden layer. Technically, the model will view each time step as a separate feature instead of separate time steps. 

We almost always have multiple samples, therefore, the model will expect the input component of training data to have the dimensions or shape: [samples, features].

The model is fit using the Adam version of *stochastic gradient* descent and optimized using the *mean squared error*, or 'mse', loss function. Once the model is defined, we can fit it on the training dataset.

In [0]:
# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_steps))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=2000, verbose=0)

Now, after the model is fit, we can of course use it and make a prediction. We can predict the next value in the sequence by providing for example this input: [16.9, 17.5, 21].

In [0]:
# demonstrate prediction
x_input = array([16.9, 17.5, 21])
x_input = x_input.reshape((1, n_steps))
yhat = model.predict(x_input, verbose=0)
print(yhat)

**Note:** Given the stochastic nature of this algorithm, your  results may vary.